## 파이썬 버전

In [1]:
import platform

platform.python_version()

'3.11.3'

## 1. 스레드 처리하기 

- 파이썬에서 스레드를 사용하여 간단한 동시성 처리를 구현하는 예제를 제공합니다. 
- 스레드는 파이썬의 threading 모듈을 통해 사용할 수 있습니다.

In [3]:
import threading
import time
import random

## 1-1 스레드 내부에 있는 속성 확인 

In [3]:
for i in dir(threading) :
    if not i.startswith("_") : 
        print(i, end=", ")

Barrier, BoundedSemaphore, BrokenBarrierError, Condition, Event, ExceptHookArgs, Lock, RLock, Semaphore, TIMEOUT_MAX, Thread, ThreadError, Timer, WeakSet, activeCount, active_count, currentThread, current_thread, enumerate, excepthook, functools, get_ident, get_native_id, getprofile, gettrace, local, main_thread, setprofile, settrace, stack_size, 

## 1-2  스레드 정의

-  클래스를 상속 받아서 클래스 정의
-  생성자에서 스레드를 생성하고 run으로 클래스를 실행 

### 스레드 클래스를 정의 

-  내부에는 run 메서드를 정의 

In [4]:
class myWorkerThread(threading.Thread) :
    def __init__(self) :
        print(" Hello Thread ")
        threading.Thread.__init__(self)
        
    def run(self) :                        # 스레드 start 메서드 호출되면 실행될 메서드 정의 
        print("Thread is now running")

### 클래스 객체를 생성

In [5]:
myThread = myWorkerThread()

 Hello Thread 


### 스레드를 실행과 종료 

In [6]:
myThread.start()
myThread.join()

Thread is now running


## 1-3  스레드 네의 특정 정보 처리 

- 생성자에 스레드 정보를 추가한다

### 클래스 정의 

In [7]:
class MyThread1(threading.Thread):
    def __init__(self, thread_id, name):
        threading.Thread.__init__(self)
        self.thread_id = thread_id
        self.name = name

    def run(self):
        print(f"Thread {self.thread_id} ({self.name}) is running")
        time.sleep(3)
        print(f"Thread {self.thread_id} ({self.name}) completed")


### 스레드 객체를 생성하고 실행하기

In [8]:
# 스레드 인스턴스 생성
thread1 = MyThread1(thread_id=1, name="Thread 1")
thread2 = MyThread1(thread_id=2, name="Thread 2")

# 스레드 시작
thread1.start()
thread2.start()

# 메인 스레드에서도 일부 작업을 수행할 수 있음
print("Main thread is running")

# 각 스레드가 완료될 때까지 대기
thread1.join()
thread2.join()

print("All threads are completed")


Thread 1 (Thread 1) is runningThread 2 (Thread 2) is running

Main thread is running
Thread 2 (Thread 2) completedThread 1 (Thread 1) completed

All threads are completed


## 1-4 데몬 스레드 처리하기

- 데몬 스레드란 종료점이 정의되지 않은 실질적인 스레드이다. 이는 프로그램이 종료될때까지 계속 작동한다.
- 데몬 스레드는 프로그램의 백그라운드에 위치해 주기적으로 변경없이 갱신 값을 전달한다



### 스레드에서 작동할 함수 정의 

In [9]:
def worker_function(name):
    for _ in range(3):
        print(f"Worker {name} is working")
        time.sleep(1)
    print(f"Worker {name} completed")

### 스레드 실행할 함수 정의  

- 스레드를 데몬으로 지정하기 

In [10]:
def main():
    # 스레드 생성
    thread1 = threading.Thread(target=worker_function, args=("A",))
    thread2 = threading.Thread(target=worker_function, args=("B",))

    # 데몬 스레드 설정 (메인 스레드가 종료되면 함께 종료됨)
    thread1.daemon = True
    thread2.daemon = True

    # 스레드 시작
    thread1.start()
    thread2.start()

    # 스레드가 완료될 때까지 대기
    thread1.join()
    thread2.join()

    print("All threads are completed")


### 스레드 실행 

In [11]:
if __name__ == "__main__":
    # 현재 스레드 정보 얻기
    current_thread = threading.current_thread()
    print(f"Current thread name: {current_thread.name}")

    # 현재 활성화된 스레드 개수 얻기
    active_threads = threading.active_count()
    print(f"Active threads: {active_threads}")

    # 메인 함수 실행
    main()


Current thread name: MainThread
Active threads: 8
Worker A is working
Worker B is working
Worker A is working
Worker B is working
Worker A is workingWorker B is working

Worker B completed
Worker A completed
All threads are completed


##  1-5 데몬스레드와 일반 스레드 처리 차이점 

### 데몬 스레드와 일반 스레드 간의 주요 차이점

- 데몬 스레드가 메인 스레드가 종료될 때 함께 종료되는 것입니다. 이를테면, 프로그램이 모든 일반 스레드의 작업을 완료하고 메인 스레드가 종료되더라도, 데몬 스레드는 메인 스레드가 종료될 때 함께 강제 종료됩니다.

-  데몬 스레드가 메인 스레드와 독립적으로 실행되는 동안, 일반 스레드는 메인 스레드에 종속되어 있어 메인 스레드가 종료되더라도 계속 실행될 수 있습니다.

### 스레드에서 실행할 함수 정의 

In [12]:
def daemon_thread():
    while True:                            ## 무한순환을 실행 중이라서 현재 작동하는 것이 계속 실행할 수 있음 
        print("Daemon thread is running")
        time.sleep(1)

def non_daemon_thread():
    for _ in range(5):
        print("Non-daemon thread is working")
        time.sleep(1)
    print("Non-daemon thread completed")

### 스레드 실행 

In [13]:
%%writefile demon.py

import threading
import time

def daemon_thread():
    while True:
        print("Daemon thread is running")
        time.sleep(1)

def non_daemon_thread():
    for _ in range(5):
        print("Non-daemon thread is working")
        time.sleep(1)
    print("Non-daemon thread completed")

if __name__ == "__main__":
    # 데몬 스레드 생성
    daemon_thread = threading.Thread(target=daemon_thread)
    daemon_thread.daemon = True  # 데몬 스레드로 설정

    # 일반 스레드 생성
    non_daemon_thread = threading.Thread(target=non_daemon_thread)

    # 스레드 시작
    daemon_thread.start()
    non_daemon_thread.start()

    try:
        # 모든 스레드가 완료될 때까지 대기
        non_daemon_thread.join()
        daemon_thread.join()
    except KeyboardInterrupt:
        # Ctrl+C 등의 인터럽트 시그널이 발생하면 프로그램 종료
        pass

    print("Main thread completed")


Overwriting demon.py


### 무한순환중에는 강제로 종료시킴 

- 데몬 스레드는 프로그램이 메인 스레드를 포함하여 모든 일반 스레드의 실행이 완료되면 자동으로 종료됩니다. 

#### 그러나 주의해야 할 중요한 점이 있습니다. 

- 프로그램이 메인 스레드가 종료될 때 모든 일반 스레드의 실행이 완료되지 않았다면, 데몬 스레드가 강제 종료될 수 있습니다.

In [14]:
%run demon.py

Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread is working
Non-daemon thread is workingDaemon thread is running

Non-daemon thread completed
Daemon thread is running
Daemon thread is running
Daemon thread is running
Daemon thread is running
Daemon thread is running
Daemon thread is running
Daemon thread is running
Daemon thread is running
Main thread completed
Main thread completed


### 위의 코드에서  데몬 스레드를 강제로 종료 처리하기

- 프로그램이 메인 스레드를 포함하여 모든 일반 스레드의 실행이 완료되면 자동으로 종료되어야 합니다. 
- 그러나 위의 코드에서는 daemon_thread 함수가 무한 루프를 갖고 있어서 종료되지 않는 문제가 발생할 수 있습니다.
- 데몬 스레드를 정상적으로 종료하려면 해당 스레드에서 실행되는 루프나 작업이 종료될 수 있도록 설계해야 합니다. 
- 루프나 작업을 완료하면 데몬 스레드는 자동으로 종료됩니다.

In [16]:


def daemon_thread2(stop_event):
    while not stop_event.is_set():
        print("Daemon thread is running")
        time.sleep(1)

def non_daemon_thread2():
    for _ in range(5):
        print("Non-daemon thread is working")
        time.sleep(1)
    print("Non-daemon thread completed")

if __name__ == "__main__":
    # 이벤트 생성
    stop_event = threading.Event()

    # 데몬 스레드 생성
    daemon_thread = threading.Thread(target=daemon_thread2, args=(stop_event,))
    daemon_thread.daemon = True  # 데몬 스레드로 설정

    # 일반 스레드 생성
    non_daemon_thread = threading.Thread(target=non_daemon_thread2)

    # 스레드 시작
    daemon_thread.start()
    non_daemon_thread.start()

    try:
        # 모든 스레드가 완료될 때까지 대기
        non_daemon_thread.join()
        stop_event.set()                # 데몬 스레드 종료를 위해 이벤트 설정
        daemon_thread.join()
    except KeyboardInterrupt:
        # Ctrl+C 등의 인터럽트 시그널이 발생하면 프로그램 종료
        pass

    print("Main thread completed")


Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread is working
Daemon thread is running
Non-daemon thread completed
Main thread completed


# 2. 스레드 활용하기 

## 2-1 여러 스레드 만들어서 실행해보기  

-  함수 정의
- 스레드 내에 함수 지정
- 스레드 시작과 조인 

### 스레드에 들어갈 함수 정의

In [1]:
def executeThread(i) :
    print(f"Thread {i} started")
    sleepTime = random.randint(1,10)
    time.sleep(sleepTime)
    print(f"Thread sleep {sleepTime} ")

### 스레드 시작 및 종료

In [4]:
for i in range(10) :
    thread = threading.Thread(target=executeThread, args=(i,))
    
    thread.start()   # 스레드 시작


print("Active Threads", threading.enumerate())
thread.join()

Thread 0 startedThread 1 started

Thread 2 started
Thread 3 started
Thread 4 started
Thread 5 started
Thread 6 started
Thread 7 started
Thread 8 started
Thread 9 started
Active Threads [<_MainThread(MainThread, started 7978636032)>, <Thread(IOPub, started daemon 6111883264)>, <Heartbeat(Heartbeat, started daemon 6128709632)>, <Thread(Thread-3 (_watch_pipe_fd), started daemon 6146682880)>, <Thread(Thread-4 (_watch_pipe_fd), started daemon 6163509248)>, <ControlThread(Control, started daemon 10754224128)>, <HistorySavingThread(IPythonHistorySavingThread, started 10771050496)>, <ParentPollerUnix(Thread-2, started daemon 10787876864)>, <Thread(Thread-5 (executeThread), started 10804703232)>, <Thread(Thread-6 (executeThread), started 10821529600)>, <Thread(Thread-7 (executeThread), started 10838355968)>, <Thread(Thread-8 (executeThread), started 10855182336)>, <Thread(Thread-9 (executeThread), started 10872008704)>, <Thread(Thread-10 (executeThread), started 10888835072)>, <Thread(Thread-11

##  2-2. 스레드별로  함수 지정해서  실행하기 

- threading.Thread를 사용하여 각 작업을 별도의 스레드로 만들고, start() 메서드로 각 스레드를 시작합니다. 
- 그리고 join() 메서드로 각 스레드가 완료될 때까지 기다립니다.

### 두 개의 작업(task1 및 task2)을 각각의 스레드에서 실행합니다.

In [5]:
def task1():
    for _ in range(5):
        print("Task 1 is running  ",threading.current_thread()," ")
        time.sleep(1)

def task2():
    for _ in range(5):
        print("Task 2 is running  ",threading.current_thread(), " ")
        time.sleep(1)

###  스레드를 2개 만들어서 실행합니다.

In [6]:
if __name__ == "__main__":
    # 두 개의 스레드 생성
    thread1 = threading.Thread(target=task1)
    thread2 = threading.Thread(target=task2)
    
    print("main thread : ", threading.main_thread())
    # 각각의 스레드 시작
    thread1.start()
    thread2.start()
    print("active count : ", threading.active_count())
    
    # 각각의 스레드가 완료될 때까지 대기
    thread1.join()
    thread2.join()

    print("Both tasks are completed")


main thread :  <_MainThread(MainThread, started 7978636032)>
Task 1 is running   <Thread(Thread-15 (task1), started 10804703232)>  
Task 2 is running   <Thread(Thread-16 (task2), started 10821529600)>  
active count :  10
Task 2 is running   <Thread(Thread-16 (task2), started 10821529600)>  
Task 1 is running   <Thread(Thread-15 (task1), started 10804703232)>  
Task 2 is running   <Thread(Thread-16 (task2), started 10821529600)>  
Task 1 is running   <Thread(Thread-15 (task1), started 10804703232)>  
Task 2 is running  Task 1 is running   <Thread(Thread-15 (task1), started 10804703232)>  
 <Thread(Thread-16 (task2), started 10821529600)>  
Task 1 is running  Task 2 is running   <Thread(Thread-16 (task2), started 10821529600)>  
 <Thread(Thread-15 (task1), started 10804703232)>  
Both tasks are completed


## 2-3. 여러 스레드간 데이터 공유

- 스레드 간에 데이터를 안전하게 전달하기 위해서는 threading 모듈에서 제공하는 Lock 및 Queue와 같은 도구를 사용할 수 있습니다. 
- 여러 스레드에서 공유되는 데이터에 대한 안전성을 보장하기 위해 동기화 메커니즘이 필요합니다.

- threading.Lock을 사용하여 데이터에 대한 접근을 동기화하고, queue.Queue를 사용하여 스레드 간 데이터 전달을 구현합니다.

### 큐 모듈을 사용 

In [7]:
import queue

### 스레드 별로 처리할 함수 정의

- 데이터를 생산하는 프로듀서와 데이터를 소비하는 컨슈머 함수 작성 


In [8]:
def producer(queue, lock):
    for i in range(5):
        time.sleep(1)
        with lock:
            print(f"Producing {i}")
            # 큐에 정보를 전달 
            queue.put(i)

def consumer(queue, lock):
    while True:
        time.sleep(1)
        with lock:
            if not queue.empty():
                # 큐에 정보를 처리
                item = queue.get()
                print(f"Consuming {item}")
            else:
                break

### 스레드를 실행 

-  큐를 만들어서 데이터 전달 공간을 만든다.
- 스레드 락을 만들어서 전달된 데이터를 동기화를 맞춘다

In [9]:

if __name__ == "__main__":
    
    # 큐 생성
    shared_queue = queue.Queue()
    
    # 스레드 락 
    shared_lock = threading.Lock()

    # Producer 스레드 시작
    producer_thread = threading.Thread(target=producer, args=(shared_queue, shared_lock))
    producer_thread.start()

    # Consumer 스레드 시작
    consumer_thread = threading.Thread(target=consumer, args=(shared_queue, shared_lock))
    consumer_thread.start()

    # 각 스레드가 완료될 때까지 대기
    producer_thread.join()
    consumer_thread.join()

    print("Both threads are completed")


Producing 0
Consuming 0
Producing 1
Producing 2
Producing 3
Producing 4
Both threads are completed


## 2-4.  스레드를 사용한 동일한 데이터 갱신 : 뮤텍스 처리 


- 스레드 간의 데이터를 안전하게 동기화하기 위해 뮤텍스(뮤텐)를 사용할 수 있습니다. 
- 뮤텍스는 락(lock)이라고도 불리며, 여러 스레드가 동시에 공유 데이터에 접근하는 것을 막기 위해 사용됩니다.

- 파이썬에서는 threading 모듈의 Lock 클래스를 사용하여 뮤텍스를 구현할 수 있습니다. 
- 뮤텍스를 사용하면 특정 코드 블록을 하나의 스레드만 실행할 수 있도록 만들 수 있습니다.

### 공유할 데이터를 전역변수로 정의 

In [10]:
# 공유 데이터
shared_data = 0

### 뮤텍스 생성  

- threading.Lock을 사용하여 modify_shared_data 함수에서 공유 데이터 shared_data를 안전하게 수정하고 있습니다. 

- with mutex: 구문을 사용하여 뮤텍스가 활성화된 동안에만 해당 코드 블록이 실행되도록 하였습니다.

In [11]:

# 뮤텍스 생성
mutex = threading.Lock()

def modify_shared_data():
    global shared_data
    with mutex:                  # 처리되는 동안 락처리 
                                 # 뮤텍스를 사용하여 공유 데이터 안전하게 수정
        for _ in range(1000):
            shared_data += 1

### 스레드를 생성하고 함수에서 동일한 데이터를 갱신한다 

In [12]:

if __name__ == "__main__":
    # 두 개의 스레드 생성
    thread1 = threading.Thread(target=modify_shared_data)
    thread2 = threading.Thread(target=modify_shared_data)

    # 각각의 스레드 시작
    thread1.start()
    thread2.start()

    # 각각의 스레드가 완료될 때까지 대기
    thread1.join()
    thread2.join()

    print("Final shared_data:", shared_data)


Final shared_data: 2000


## 2-5.  이벤트 처리 

- 스레드를 사용하여 이벤트 루프를 처리하려면 threading 모듈과 이벤트 객체를 활용할 수 있습니다.
- 이벤트 객체는 스레드 간에 상태를 전달하고 스레드가 특정 조건을 기다리도록 하는 데 사용됩니다.

### 주의사항:

- threading.Event 객체를 사용하여 이벤트를 생성하고, is_set() 메서드로 이벤트가 설정되었는지 확인합니다.
- time.sleep 함수를 사용하여 스레드가 루프를 실행하는 동안 일시적으로 대기하도록 합니다.
- KeyboardInterrupt 예외를 처리하여 사용자가 프로그램을 중지하면 이벤트를 설정하고 스레드를 종료합니다.
- thread.join()을 사용하여 스레드가 완료될 때까지 기다립니다.

In [13]:
# 이벤트 객체 생성
event = threading.Event()

In [14]:
def event_loop():
    while not event.is_set():  # 이벤트가 설정되지 않은 동안 계속 반복
        print("Event loop is running")
        time.sleep(1)

In [15]:

if __name__ == "__main__":
    # 이벤트 루프를 처리할 스레드 생성
    thread = threading.Thread(target=event_loop)

    # 스레드 시작
    thread.start()

    try:
        # 일정 시간 동안 대기
        time.sleep(5)

        # 이벤트 설정
        event.set()

        # 스레드가 완료될 때까지 대기
        thread.join()
    except KeyboardInterrupt:
        # KeyboardInterrupt 예외 발생 시, 이벤트 설정 및 스레드 종료
        event.set()
        thread.join()

    print("Event loop completed")


Event loop is running
Event loop is running
Event loop is running
Event loop is running
Event loop is running
Event loop completed
